In [140]:
import torch
import torch.nn as nn

In [141]:
class Layer:
    def __init__(self,weight, bias) :
        self.weight = weight
        self.bias = bias
        
    def activation(self,input, func) :
        return func(input)

    def forward(self, input, func) :
        output = torch.matmul(input,self.weight)
        if (self.bias is not None):
            output += self.bias
        return self.activation(output, func)
    
    def evaluate(self, pred, target) : #MSE
        return nn.MSELoss()(pred,target)

In [142]:
class Network:
    def __init__ (self, batchSize, weights, biases, totalLayer, forwards, learningRate, epoch):
        self.batchSize = batchSize #ukuran batch
        self.learningRate = learningRate
        self.epoch = epoch
        self.totalLayer = totalLayer #total layer yang digunakan
        self.forwards = forwards #buat nentuin pake forward function apa aja pada setiap layer
        self.layers = [] #list of layers
        self.functions = [self.reLU, self.sigmoid, self.tanh, self.softmax] #list of function, kalo di layer 1 mau pake sigmoid, maka di self.functions[0] = 1
        for i in range (0, totalLayer):
            self.layers.append(Layer(weights[i], biases[i])) #layer ke i disimpan dalam self.layers[i]
    
    def forward (self, input):
        for i in range (0, self.totalLayer):
          input = self.layers[i].forward(input, self.functions[self.forwards[i]])
        return input

    def backward (self, loss):
        loss.backward()
        for i in range (0, self.totalLayer):
            with torch.no_grad():
                self.layers[i].weight += self.learningRate * self.layers[i].weight.grad
            self.layers[i].weight.grad.zero_()
    
    def singleProcessing (self, input, target):
        pred = self.forward(input)
        loss = self.evaluate(pred, target)
        self.backward(loss)

    def batchProcessing (self, input, target):
        wholeInput = torch.split(input, self.batchSize)
        wholeTarget = torch.split(target, self.batchSize)
        sz = len(wholeInput)
        for i in range (0, sz):
            self.singleProcessing(wholeInput[i], wholeTarget[i])

    def printLoss (self, input, target):
        wholeInput = torch.split(input, self.batchSize)
        wholeTarget = torch.split(target, self.batchSize)
        sz = len(wholeInput)
        for i in range (0, sz):
            pred = self.forward(wholeInput[i])
            print(self.evaluate(pred, wholeTarget[i]))

    def startLearning (self, input, target):
        for i in range (0, self.epoch):
            self.batchProcessing(input, target)
        self.printLoss(input, target)
        
    def reLU(self, input):
        return nn.ReLU()(input)

    def sigmoid(self, input):
        return nn.Sigmoid()(input)

    def tanh(self, input):
        return nn.Tanh()(input)

    def softmax(self, input):
        return nn.Softmax(dim=0)(input)
    
    def evaluate(self, pred, target):
        return nn.MSELoss()(pred,target)

In [143]:
# COBA COBA AJA

input = torch.Tensor([[-1.1258398,  -1.1523602],
                      [-0.25057858, -0.4338788],
                      [0.84871036,  0.69200915],
                      [-0.31601277, -2.1152194],
                      [0.32227492, -1.2633348],
                      [0.3499832,  0.30813393],
                      [0.11984151,  1.2376579],
                      [1.1167772,  -0.24727815],
                      [-1.3526537,  -1.6959312],
                      [0.5666506,   0.79350835],
                      [0.59883946, -1.5550951],
                      [-0.3413604,   1.8530061],
                      [-0.21586326, -0.74254817],
                      [0.5627214,   0.2596274],
                      [-0.173961,   -0.6787462],
                      [0.93826073,  0.48886982],
                      [1.2032237,   0.0845347],
                      [-1.2001394,  -0.00478574],
                      [-0.5180748,  -0.3067042],
                      [-1.5809939,   1.7066433]])

target = torch.Tensor([[0.20552567, -0.45032975],
                       [-0.5730771,  -0.5553584],
                       [0.59432304,  1.5419426],
                       [0.5073344,  -0.59103316],
                       [-1.325326,    0.18855357],
                       [-0.06907269, -0.49492535],
                       [-1.4959149,  -0.19383712],
                       [0.44551218,  1.3252748],
                       [1.5091219,   2.0819554],
                       [1.7067116,   2.3803675],
                       [-1.1256016, -0.3169981],
                       [-1.0924683,  -0.0851943],
                       [1.645867,   -1.3601689],
                       [0.34456542,  0.5198677],
                       [-2.6133225,  -1.6964744],
                       [-0.22824179,  0.279955],
                       [0.2469264,  0.076887],
                       [0.3380058,   0.45440176],
                       [0.45694014, -0.86537135],
                       [0.78130794, -0.9267894]])

weights = [

    torch.tensor([[-1.5054897,  -0.66098255,  1.3232017,   0.0371143,  -0.2849093],
                 [-0.13344175,  1.8929105,   3.1110442,  -0.4583958,  -0.33598807]], requires_grad = True),

    torch.tensor([[-1.7342502,  -1.3360486,   0.88709605],
                  [0.76795745,  0.057113,    0.22395839],
                  [0.5519643,  -0.578798,    0.01772212],
                  [0.13182636,  1.0197668,  -0.4468389],
                  [0.45202246, -0.97592443,  0.7112372]], requires_grad = True),

    torch.tensor([[-0.15909262, -1.7786636],
                  [0.84765124,  0.24594283],
                  [-0.13116787, -0.17851807]], requires_grad = True)

]


biases = [
    torch.tensor([[-1.5699861,   1.2315004,   1.3946317,   1.1711024,   0.43351194]]
                 ),
    torch.tensor([[-0.75822645, -0.64358306, -0.6461524]]),
    torch.tensor([[-0.5958931,   0.27386975]])
]

model = Network(5, weights, biases, 3, [2, 2, 3], 5, 100)
model.startLearning(input, target)


tensor(1.1049, grad_fn=<MseLossBackward0>)
tensor(1.6469, grad_fn=<MseLossBackward0>)
tensor(2.4009, grad_fn=<MseLossBackward0>)
tensor(0.6261, grad_fn=<MseLossBackward0>)


In [149]:
# dataset and testing

torch.manual_seed(87)

from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

images = x_train[0:1000].reshape(1000,28*28) / 255
labels = y_train[0:1000]

